In [ ]:
# !pip uninstall -y numpy
# !pip install numpy==1.26.4 

In [ ]:
import numpy as np
import xarray as xr
import rioxarray
import glob
import matplotlib.pyplot as plt
import dask.distributed
import odc.geo.xr

In [ ]:
client = dask.distributed.Client()
client

In [ ]:
!du -h --max-depth=1 output/full_mask/

In [ ]:
mask_ds = xr.open_zarr("output/full_mask/global_modis_snow_mask.zarr",chunks='auto')#.sel(y=slice(-8885139,8885139))
#mask_ds.rio.write_crs('+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs', inplace=True)
mask_ds

In [ ]:
mask_ds['max_consec_snow_days'].sel(WY=2020).rio.to_raster("test_max_consec_snow_days_WY2020.tif", driver='COG',windowed=True)

In [ ]:
import pyproj
import xarray as xr

# Assuming mask_ds is your xarray dataset
# Define the bounding box in its original CRS (e.g., EPSG:4326)
#bbox_original_crs = {'left': -120, 'bottom': 30, 'right': -110, 'top': 40}
bbox_original_crs = {'left': -179.8, 'bottom': -80, 'right': 179.8, 'top': 80}


# Define the CRS for the bounding box and the dataset
crs_original = pyproj.CRS('EPSG:4326')  # Example: WGS84
crs_target = pyproj.CRS(mask_ds.rio.crs)  # Assuming mask_ds has a CRS defined
#crs_target = pyproj.CRS('+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs')  # Assuming mask_ds has a CRS defined
#crs_target = pyproj.CRS(modis_grid.crs)  # Assuming mask_ds has a CRS defined


# Create a transformer to convert between the original and target CRS
transformer = pyproj.Transformer.from_crs(crs_original, crs_target, always_xy=True)

# Transform the bounding box coordinates to the dataset's CRS
left, bottom = transformer.transform(bbox_original_crs['left'], bbox_original_crs['bottom'])
right, top = transformer.transform(bbox_original_crs['right'], bbox_original_crs['top'])

# Use the transformed bounding box to slice the dataset
# Note: The slicing dimensions ('x', 'y') might need to be adjusted based on your dataset's coordinate names
subset = mask_ds.sel(x=slice(left,right),y=slice(bottom, top))#.rio.reproject(crs_original).sel(x=slice(bbox_original_crs['left'], bbox_original_crs['right']), y=slice(bbox_original_crs['top'], bbox_original_crs['bottom']))

In [ ]:
left, right, bottom, top

In [ ]:
subset['max_consec_snow_days'].sel(WY=2015).rio.to_raster("test_max_consec_snow_days_WY2015.tif", driver='COG')

In [ ]:
# just host in the modis projection, use above to reproject right to utm. reprojection twice is inaccurate
#subset.odc.reproject(crs_original,resampling='nearest').to_zarr('output/full_mask_4326/global_modis_snow_mask_4326.zarr',mode='w')

In [ ]:
#subset.sel(x=slice(-120, -110), y=slice(30, 40))

In [ ]:
f,ax=plt.subplots()

subset['max_consec_snow_days'].sel(WY=2020).plot.imshow(ax=ax)

In [ ]:
import zarr
import azure.storage.blob

sas_token = 
def upload_zarr(file_path):
    container_client = azure.storage.blob.ContainerClient(
        "https://snowmelt.blob.core.windows.net",
        container_name="snowmelt",
        credential=sas_token,
    )
    store = zarr.ABSStore(client=container_client, prefix='/')
    ds = zarr.open(file_path)
    group = zarr.group(store=store)
    zarr.convenience.copy(ds, group, 'sample.zarr')
print("finished writing to Azure")

In [ ]:
#mask_ds.rio.clip_box(-180,-90,180,90, crs="EPSG:4326")

In [ ]:
mask_ds.odc.reproject(
    "EPSG:4326"
)  # .to_zarr('output/full_mask/global_modis_snow_mask_4326.zarr', mode='w')

In [ ]:
mask_ds["max_consec_snow_days"].sel(WY=2020).rio.reproject(
    "EPSG:4326"
)  # .plot.imshow()

In [ ]:
import rioxarray
import xarray as xr
from osgeo import gdal
import numpy as np

# Open the Zarr dataset
ds = xr.open_zarr("output/full_mask/global_modis_snow_mask.zarr")

# Assuming the main data variable is named 'snow_mask'. Adjust if different.
data_var = ds

# Create a GDAL dataset from the xarray DataArray
gdal_dataset = gdal.Open(
    data_var.rio.write_crs(data_var.rio.crs, inplace=True).rio.to_vrt(),
    gdal.GA_ReadOnly,
)

# Define source and target projections
src_proj = (
    "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
)
dst_proj = "EPSG:4326"

# Create a VRT with the new projection
vrt_options = gdal.BuildVRTOptions(outputBounds=None, outputBoundsSRS=dst_proj)
vrt = gdal.BuildVRT("", [gdal_dataset], options=vrt_options)

# Set the projection on the VRT
vrt.SetProjection(dst_proj)

# Create a warped VRT
warped_vrt = gdal.AutoCreateWarpedVRT(vrt, src_proj, dst_proj)

# Read the data from the warped VRT
warped_data = warped_vrt.ReadAsArray()

# Get the geotransform and projection of the warped data
geotransform = warped_vrt.GetGeoTransform()
projection = warped_vrt.GetProjection()

# Create a new xarray DataArray with the warped data
new_coords = {
    "y": np.arange(warped_data.shape[1]) * geotransform[5] + geotransform[3],
    "x": np.arange(warped_data.shape[2]) * geotransform[1] + geotransform[0],
}
new_da = xr.DataArray(warped_data, coords=new_coords, dims=("band", "y", "x"))

# Set the CRS attribute
new_da.rio.write_crs(dst_proj, inplace=True)

# Create a new Dataset with the warped DataArray
new_ds = new_da.to_dataset(name="snow_mask")

# Save the new Dataset to a Zarr store
new_ds.to_zarr("/path/to/output_reprojected.zarr")

# Clean up
del vrt, warped_vrt, gdal_dataset